<a href="https://colab.research.google.com/github/ThrishaJawahar/Falcon-9-Launch-Prediction/blob/main/EDAwithSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



1.   Understand the Spacex DataSet
2.   Load the dataset into the corresponding table in a Db2 database







In [ ]:
!pip install sqlalchemy==1.3.9
!pip install ipython-sql
!pip install pandas


  Using cached SQLAlchemy-1.3.9-cp310-cp310-linux_x86_64.whl
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.29.0 requires sqlalchemy<3.0dev,>=1.4, but you have sqlalchemy 1.3.9 which is incompatible.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.3.9 which is incompatible.
langchain 0.3.12 requires SQLAlchemy<3,>=1.4, but you have sqlalchemy 1.3.9 which is incompatible.
  Using cached SQLAlchemy-2.0.36-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.7 kB)
Using cached SQLAlchemy-2.0.36-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.9

In [1]:
from google.colab import files
import pandas as pd
import sqlite3
import io
%load_ext sql


In [2]:

uploaded = files.upload()


Saving Spacex.csv to Spacex.csv


In [4]:
df = pd.read_csv(io.BytesIO(uploaded['Spacex.csv']))
df.head()

,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [11]:
import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'

con = sqlite3.connect("my_data1.db")
cur = con.cursor()
print(con)

In [6]:
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method="multi")

101

In [13]:
%sql sqlite:///my_data1.db
%sql SELECT * FROM SPACEXTABLE LIMIT 5

 * sqlite:///my_data1.db
(sqlite3.OperationalError) no such table: SPACEXTABLE
[SQL: SELECT * FROM SPACEXTABLE LIMIT 5]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [14]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

Task 1
Display the names of the unique launch sites in the space mission

In [15]:
%sql SELECT DISTINCT "Launch_Site" FROM SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


Task 2
Display 5 records where launch sites begin with the string 'CCA'

In [16]:
%sql SELECT * FROM SPACEXTABLE WHERE "Launch_Site" LIKE 'CCA%' LIMIT 5;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


Task 3
Display the total payload mass carried by boosters launched by NASA (CRS)

In [17]:
sql SELECT SUM("PAYLOAD_MASS__KG_") AS total_payload_mass FROM SPACEXTABLE WHERE "Customer" = 'NASA (CRS)';

 * sqlite:///my_data1.db
Done.


total_payload_mass
45596


Task 4
Display average payload mass carried by booster version F9 v1.1

In [18]:
%sql SELECT AVG("PAYLOAD_MASS__KG_") AS avg_payload_mass FROM SPACEXTABLE WHERE "Booster_Version" = 'F9 v1.1';


 * sqlite:///my_data1.db
Done.


avg_payload_mass
2928.4


Task 5
List the date when the first succesful landing outcome in ground pad was acheived.

In [19]:
%sql SELECT MIN("Date") AS first_successful_landing_on_ground_pad FROM SPACEXTABLE WHERE "Landing_Outcome" = 'True RTLS';


 * sqlite:///my_data1.db
Done.


first_successful_landing_on_ground_pad
None


Task 6
List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000

In [20]:
%sql SELECT DISTINCT "Booster_Version" FROM SPACEXTABLE WHERE "Landing_Outcome" = 'True ASDS' AND "PAYLOAD_MASS__KG_" > 4000 AND "PAYLOAD_MASS__KG_" < 6000;


 * sqlite:///my_data1.db
Done.


Booster_Version


Task 7
List the total number of successful and failure mission outcomes


In [21]:
%sql SELECT "Mission_Outcome", COUNT(*) FROM SPACEXTABLE GROUP BY "Mission_Outcome";


 * sqlite:///my_data1.db
Done.


Mission_Outcome,COUNT(*)
Failure (in flight),1
Success,98
Success,1
Success (payload status unclear),1


Task 8
List the names of the booster_versions which have carried the maximum payload mass. Use a subquery

In [22]:
%sql SELECT "Booster_Version" FROM SPACEXTABLE WHERE "PAYLOAD_MASS__KG_" = (SELECT MAX("PAYLOAD_MASS__KG_") FROM SPACEXTABLE);

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


Task 9
List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

In [23]:
%sql SELECT \
CASE \
WHEN substr("Date", 6, 2) = '01' THEN 'January'\
WHEN substr("Date", 6, 2) = '02' THEN 'February'\
WHEN substr("Date", 6, 2) = '03' THEN 'March'\
WHEN substr("Date", 6, 2) = '04' THEN 'April'\
WHEN substr("Date", 6, 2) = '05' THEN 'May'\
WHEN substr("Date", 6, 2) = '06' THEN 'June'\
WHEN substr("Date", 6, 2) = '07' THEN 'July'\
WHEN substr("Date", 6, 2) = '08' THEN 'August'\
WHEN substr("Date", 6, 2) = '09' THEN 'September'\
WHEN substr("Date", 6, 2) = '10' THEN 'October'\
WHEN substr("Date", 6, 2) = '11' THEN 'November'\
WHEN substr("Date", 6, 2) = '12' THEN 'December'\
ELSE 'Unknown'\
END AS Month,\
"Landing_Outcome", \
"Booster_Version", \
"Launch_Site"\
FROM SPACEXTABLE\
WHERE substr("Date", 1, 4) = '2015' \
AND "Landing_Outcome" = 'Failure (Drone Ship)';


 * sqlite:///my_data1.db
Done.


Month,Landing_Outcome,Booster_Version,Launch_Site


Task 10
Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.

In [37]:
%sql SELECT "Landing_Outcome", COUNT(*) AS "Landing_Outcome_Count" \
     FROM SPACEXTABLE \
     WHERE "Date" BETWEEN '2010-06-04' AND '2017-03-20' \
     GROUP BY "Landing_Outcome" \
     ORDER BY "Landing_Outcome_Count" DESC;


 * sqlite:///my_data1.db
Done.


Landing_Outcome,Landing_Outcome_Count
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1
